In [0]:
#Getting Secret Scope created 
dbutils.secrets.get(scope = "data-election-key", key = "data-election-app-secret" )

Out[93]: '[REDACTED]'

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "##########################", #provide your Client ID
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "data-election-key", key = "data-election-app-secret" ),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/################################/oauth2/token"} #provide your Tenant ID

# Optionally, you can add <directory-name> to the source URI of your mount point.
# container@storage
# storage> add role > app-reg. 
dbutils.fs.mount(
  source = "abfss://data-election-brazil-container@dataelectionstorage.dfs.core.windows.net",
  mount_point = "/mnt/data-election",
  extra_configs = configs)

Out[8]: True

In [0]:
%fs ls /mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/


path,name,size,modificationTime
dbfs:/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_BRASIL.csv,consulta_cand_2022_BRASIL.csv,22784583,1707327409000


In [0]:
# import libs
import os
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.functions import to_timestamp


In [0]:
#Getting files path to delete except the endswith "BRASIL.csv" (national for president)

directory = "/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377"
[os.path.join(directory, filename) for filename in os.listdir(directory) if not filename.endswith("BRASIL.csv")]


Out[3]: ['/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_AC.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_AL.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_AM.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_AP.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_BA.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_BR.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_CE.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_DF.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_ES.csv',
 '/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_G

In [0]:
#Deleting files except "BRASIL.csv"

# List names of files in the directory
files = os.listdir('/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377')

# For loop to remove files except Brasil.csv
for file in files:
    if not file.endswith("BRASIL.csv"):
        file_path = os.path.join('/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377', file)
        #print(file_path) #print to check if getting the right files except "BRASIL.csv"
        os.remove(file_path)

In [0]:
#Check if the file "BRASIL.csv" is there. OK.
files = os.listdir('/dbfs/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377')
print(files)

['consulta_cand_2022_BRASIL.csv']


In [0]:
df = (spark.read
      .format("csv")
      .options(**{"sep":";",
                  "header": "true",
                  "encoding": "latin1"})
      .load("/mnt/data-election/raw-data/879640fa-29fa-4155-82b7-ec38b63b8377/consulta_cand_2022_BRASIL.csv"))

df.limit(10).display()

DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,NM_TIPO_DESTINACAO_VOTOS,CD_SITUACAO_CANDIDATO_TOT,DS_SITUACAO_CANDIDATO_TOT,ST_PREST_CONTAS,ST_SUBSTITUIDO,SQ_SUBSTITUIDO,SQ_ORDEM_SUPLENCIA,DT_ACEITE_CANDIDATURA
06/02/2024,18:38:19,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,TO,TO,TOCANTINS,7,DEPUTADO ESTADUAL,270001697206,12121,FERNANDO DOURADO DA SILVA,FERNANDO DOURADO,#NULO#,02108422102,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,-1,#NULO#,#NULO#,#NULO#,270001684650,PARTIDO ISOLADO,PDT,1,BRASILEIRA NATA,MA,-3,IMPERATRIZ,21/03/1988,34,037379252798,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),03,PARDA,257,EMPRESÁRIO,1270629.01,5,SUPLENTE,N,S,-1,06007433320226270000,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,15,14/08/2022 15:43:16
06/02/2024,18:38:19,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,DF,DF,DISTRITO FEDERAL,9,1º SUPLENTE,70001651531,456,MARIA DE LOURDES WERNECK,LOURDINHA WERNECK,#NULO#,01068199687,NÃO DIVULGÁVEL,3,INAPTO,6,RENÚNCIA,FEDERAÇÃO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,1,Federação PSDB Cidadania,PSDB/CIDADANIA,PSDB/CIDADANIA,70001683352,FEDERAÇÃO,PSDB/CIDADANIA,1,BRASILEIRA NATA,MG,-3,DIVINÓPOLIS,31/12/1941,81,000309382097,4,FEMININO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),01,BRANCA,257,EMPRESÁRIO,-1,-1,#NULO#,N,S,-1,06011032320226070000,-1,#NULO#,-1,#NULO#,NÃO,#NULO#,-1,#NULO#,N,S,-1,-1,12/08/2022 17:07:39
06/02/2024,18:38:19,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,RO,RO,RONDÔNIA,7,DEPUTADO ESTADUAL,220001609406,11755,ANA CRISTINA MONTEIRO ROCHA BARROS,ANA BARROS,#NULO#,84894458268,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,11,PP,PROGRESSISTAS,-1,#NULO#,#NULO#,#NULO#,220001681436,PARTIDO ISOLADO,PP,1,BRASILEIRA NATA,RO,-3,PORTO VELHO,09/05/1983,39,013130132372,4,FEMININO,3,ENSINO FUNDAMENTAL INCOMPLETO,3,CASADO(A),01,BRANCA,999,OUTROS,1270629.01,5,SUPLENTE,N,S,-1,06004795420226220000,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,20,08/08/2022 11:47:03
06/02/2024,18:38:19,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,SP,SP,SÃO PAULO,6,DEPUTADO FEDERAL,250001620739,5142,FERNANDO CHIARELLI,FERNANDO CHIARELLI,#NULO#,86385445887,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,51,PATRIOTA,PATRIOTA,-1,#NULO#,#NULO#,#NULO#,250001682231,PARTIDO ISOLADO,PATRIOTA,1,BRASILEIRA NATA,SP,-3,RIBEIRÃO PRETO,21/01/1957,66,172560920167,2,MASCULINO,2,LÊ E ESCREVE,3,CASADO(A),01,BRANCA,999,OUTROS,3176572.53,4,NÃO ELEITO,N,N,-1,06024990720226260000,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,-1,10/08/2022 15:54:40
06/02/2024,18:38:19,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,PB,PB,PARAÍBA,3,GOVERNADOR,150001718119,22,NILVAN FERREIRA DO NASCIMENTO,NILVAN FERREIRA,#NULO#,91832608468,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,22,PL,PARTID

In [0]:
df.printSchema()

root
 |-- DT_GERACAO: string (nullable = true)
 |-- HH_GERACAO: string (nullable = true)
 |-- ANO_ELEICAO: string (nullable = true)
 |-- CD_TIPO_ELEICAO: string (nullable = true)
 |-- NM_TIPO_ELEICAO: string (nullable = true)
 |-- NR_TURNO: string (nullable = true)
 |-- CD_ELEICAO: string (nullable = true)
 |-- DS_ELEICAO: string (nullable = true)
 |-- DT_ELEICAO: string (nullable = true)
 |-- TP_ABRANGENCIA: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- SG_UE: string (nullable = true)
 |-- NM_UE: string (nullable = true)
 |-- CD_CARGO: string (nullable = true)
 |-- DS_CARGO: string (nullable = true)
 |-- SQ_CANDIDATO: string (nullable = true)
 |-- NR_CANDIDATO: string (nullable = true)
 |-- NM_CANDIDATO: string (nullable = true)
 |-- NM_URNA_CANDIDATO: string (nullable = true)
 |-- NM_SOCIAL_CANDIDATO: string (nullable = true)
 |-- NR_CPF_CANDIDATO: string (nullable = true)
 |-- NM_EMAIL: string (nullable = true)
 |-- CD_SITUACAO_CANDIDATURA: string (nullable = tr

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS candidatos_2022_medallion;

In [0]:
%sql

CREATE TABLE IF NOT EXISTS candidatos_2022_medallion.candidatos_bronze_table (
    DT_GERACAO DATE,
    HH_GERACAO STRING,
    ANO_ELEICAO INTEGER,
    CD_TIPO_ELEICAO INTEGER,
    NM_TIPO_ELEICAO STRING,
    NR_TURNO INTEGER,
    CD_ELEICAO INTEGER,
    DS_ELEICAO STRING,
    DT_ELEICAO DATE,
    TP_ABRANGENCIA STRING,
    SG_UF STRING,
    SG_UE STRING,
    NM_UE STRING,
    CD_CARGO INTEGER,
    DS_CARGO STRING,
    SQ_CANDIDATO STRING,
    NR_CANDIDATO STRING,
    NM_CANDIDATO STRING,
    NM_URNA_CANDIDATO STRING,
    NM_SOCIAL_CANDIDATO STRING,
    NR_CPF_CANDIDATO STRING,
    NM_EMAIL STRING,
    CD_SITUACAO_CANDIDATURA INTEGER,
    DS_SITUACAO_CANDIDATURA STRING,
    CD_DETALHE_SITUACAO_CAND INTEGER,
    DS_DETALHE_SITUACAO_CAND STRING,
    TP_AGREMIACAO STRING,
    NR_PARTIDO INTEGER,
    SG_PARTIDO STRING,
    NM_PARTIDO STRING,
    NR_FEDERACAO STRING,
    NM_FEDERACAO STRING,
    SG_FEDERACAO STRING,
    DS_COMPOSICAO_FEDERACAO STRING,
    SQ_COLIGACAO STRING,
    NM_COLIGACAO STRING,
    DS_COMPOSICAO_COLIGACAO STRING,
    CD_NACIONALIDADE INTEGER,
    DS_NACIONALIDADE STRING,
    SG_UF_NASCIMENTO STRING,
    CD_MUNICIPIO_NASCIMENTO INTEGER,
    NM_MUNICIPIO_NASCIMENTO STRING,
    DT_NASCIMENTO DATE,
    NR_IDADE_DATA_POSSE INTEGER,
    NR_TITULO_ELEITORAL_CANDIDATO STRING,
    CD_GENERO INTEGER,
    DS_GENERO STRING,
    CD_GRAU_INSTRUCAO INTEGER,
    DS_GRAU_INSTRUCAO STRING,
    CD_ESTADO_CIVIL INTEGER,
    DS_ESTADO_CIVIL STRING,
    CD_COR_RACA STRING,
    DS_COR_RACA STRING,
    CD_OCUPACAO INTEGER,
    DS_OCUPACAO STRING,
    VR_DESPESA_MAX_CAMPANHA FLOAT,
    CD_SIT_TOT_TURNO INTEGER,
    DS_SIT_TOT_TURNO STRING,
    ST_REELEICAO STRING,
    ST_DECLARAR_BENS STRING,
    NR_PROTOCOLO_CANDIDATURA STRING,
    NR_PROCESSO STRING,
    CD_SITUACAO_CANDIDATO_PLEITO INTEGER,
    DS_SITUACAO_CANDIDATO_PLEITO STRING,
    CD_SITUACAO_CANDIDATO_URNA INTEGER,
    DS_SITUACAO_CANDIDATO_URNA STRING,
    ST_CANDIDATO_INSERIDO_URNA STRING,
    NM_TIPO_DESTINACAO_VOTOS STRING,
    CD_SITUACAO_CANDIDATO_TOT INTEGER,
    DS_SITUACAO_CANDIDATO_TOT STRING,
    ST_PREST_CONTAS STRING,
    ST_SUBSTITUIDO STRING,
    SQ_SUBSTITUIDO STRING,
    SQ_ORDEM_SUPLENCIA INTEGER,
    DT_ACEITE_CANDIDATURA TIMESTAMP)
;

In [0]:
# Correction date columns from the dataframe
bronzecandidatos = df = df.withColumn("DT_GERACAO", to_date("DT_GERACAO", "dd/MM/yyyy"))\
        .withColumn("DT_NASCIMENTO", to_date("DT_NASCIMENTO", "dd/MM/yyyy"))\
        .withColumn("DT_ELEICAO", to_date("DT_ELEICAO", "dd/MM/yyyy"))\
        .withColumn("ANO_ELEICAO", col("ANO_ELEICAO").cast("integer"))\
        .withColumn("DT_ACEITE_CANDIDATURA", to_timestamp("DT_ACEITE_CANDIDATURA", "dd/MM/yyyy HH:mm:ss"))


In [0]:
#Insert the dataframe into the table created.
bronzecandidatos.write.insertInto("candidatos_2022_medallion.candidatos_bronze_table")

In [0]:
%sql
select 
NR_TURNO
,count(NR_TURNO)
from candidatos_2022_medallion.candidatos_bronze_table
group by NR_TURNO;

NR_TURNO,count(NR_TURNO)
1,29262
2,52


In [0]:

# Calculating the Average by Gender
avg_age_by_gender = bronzecandidatos.withColumn("AGE", year("DT_ELEICAO") - year("DT_NASCIMENTO"))\
    .select("NM_CANDIDATO", "DS_GENERO", "AGE")\
    .distinct()\
    .groupBy("DS_GENERO").avg("AGE")

# Show the result
avg_age_by_gender.show()



+--------------+------------------+
|     DS_GENERO|          avg(AGE)|
+--------------+------------------+
|NÃO DIVULGÁVEL|              null|
|     MASCULINO|49.444102617582644|
|      FEMININO|46.978249770244055|
+--------------+------------------+



In [0]:
# Writing the transformed data into ADSL Gen2
bronzecandidatos.write.mode("overwrite")\
    .option("header", "true")\
    .csv("/mnt/data-election/transformed-data/candidatos-2022")